<a href="https://colab.research.google.com/github/JaredStemper/GoogleDriveMigrator/blob/main/googleDriveMigrationScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


In [4]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
#authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#will fail on google forms
MIMETYPES = {
        # Drive Document files as MS dox
        'application/vnd.google-apps.document': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        # Drive presentation as MS pptx
        'application/vnd.google-apps.presentation': 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
        # see https://developers.google.com/drive/v3/web/mime-types
    }
EXTENSIONS = {
        'application/vnd.google-apps.document': '.docx',
        'application/vnd.google-apps.spreadsheet': '.xlsx',
        'application/vnd.google-apps.presentation': '.pptx'
}

In [ ]:
def escape_fname(name):
    return name.replace('/','_')

def create_folder(path,name):
    os.mkdir('{}{}'.format(path,escape_fname(name)))

"""
get the id by going to the folder you want to copy to/from, and grabbing the last section of text from the web address

e.g. with https://drive.google.com/drive/u/2/folders/190-923045f9srOeE3zHpPmUxK1wOT
     the id of the folder is '190-923045f9srOeE3zHpPmUxK1wOT'

You will need to do this with both the source and destination folder.
"""
folder_id = 'TODO'
root_id = 'TODO'
root = '/gdrive/MyDrive/drive_download' #make sure to change "drive_download" to the name of your destination folder if it differs

In [16]:
def search_folder(folder_id, root):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % folder_id}).GetList()
    root_file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % root_id}).GetList()
    skipFolderList = [] #optional variable to include the name of any folders that shouldn't be copied
    for file in file_list:
        # print('title: %s, id: %s, kind: %s' % (file['title'], file['id'], file['mimeType']))
        # print(file)
        if file['mimeType'].split('.')[-1] == 'folder':
            foldername = escape_fname(file['title'])
            #avoid any unnecessary folders
            if foldername in skipFolderList:
              continue
            print(root,foldername)
            try:
                create_folder(root,foldername)
            except FileExistsError:
              continue
            search_folder(file['id'], '{}{}/'.format(root,foldername))
        else:
            download_mimetype = None
            filename = escape_fname(file['title'])
            filename = '{}{}'.format(root,filename)
            try:
                print('DOWNLOADING:', filename)
                if file['mimeType'] in MIMETYPES:
                   download_mimetype = MIMETYPES[file['mimeType']]
                   file.GetContentFile(filename+EXTENSIONS[file['mimeType']], mimetype=download_mimetype)
                else:
                   file.GetContentFile(filename)
            except FileExistsError:
              continue
            except:
              print('FAILED:', filename)
              f.write(filename+'\n')

with open("failed.txt","w+") as failed:
  search_folder(folder_id,root+'/')